## SWQ Seminar
### Near Duplicate Detection with Multi Modal Large Language Model

# Mount Google Drive

In [1]:
from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/SWQ'

# This will mount your google drive under 'MyDrive'
drive.mount('/content/gdrive')
# In order to access the files in this notebook we have to navigate to the correct folder
os.chdir(gdrive_path)
# Check manually if all files are present
print(sorted(os.listdir()))

Mounted at /content/gdrive
['addressbook.csv', 'addressbook.gsheet', 'labeled-subject-set', 'mycode', 'test']


# Imports

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,757 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,078 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main 

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
from PIL import Image, ImageDraw
from IPython.display import display
import requests
import time
import base64

import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from mycode.util import ScreenshotTaker, get_thumbnail_for_llava, concat_images_with_line

import pandas as pd

from tqdm import tqdm

%matplotlib inline

# Preprocessing of web apps

In [4]:
all_states_df = pd.read_csv("labeled-subject-set/SS.csv")

In [ ]:
screenshot_taker = ScreenshotTaker()

directory = "labeled-subject-set/GroundTruthModels-SS/addressbook"

for filename in tqdm(os.listdir(directory)):
  if filename.endswith(".html"):

    file_path = os.path.join(directory, filename)

    # Extract directory and file parts
    filename_with_extension = os.path.basename(file_path)
    filename_without_extension, file_extension = os.path.splitext(filename_with_extension)

    # Construct the new file path
    save_to_path = os.path.join(directory, "screenshots", f"{filename_without_extension}.png")

    screenshot_taker.take_screenshot(file_path, save_to_path)
    get_thumbnail_for_llava(save_to_path)

screenshot_taker.close()

100%|██████████| 525/525 [01:03<00:00,  8.30it/s]


In [5]:
addressbook_df = all_states_df[all_states_df['appname'] == 'addressbook']
addressbook_df

,appname,crawl,state1,state2,DOM_RTED,DOM_Levenshtein,DOM_contentHash,DOM_SIMHASH,VISUAL_BlockHash,VISUAL_PHash,VISUAL_Hyst,VISUAL_PDiff,VISUAL_SIFT,VISUAL_SSIM,HUMAN_CLASSIFICATION,TAGS
0,addressbook,crawl-addressbook-60min,index,state3,0.214286,0.148426,250.0,0.0,5.0,0.0,3.327536e+05,0.005017,54.166667,0.973036,2,NaN
1,addressbook,crawl-addressbook-60min,index,state5,0.622449,0.294496,329.0,12.0,44.0,34.0,1.690174e+07,0.149702,6.788512,0.877429,2,NaN
2,addressbook,crawl-addressbook-60min,index,state6,0.618182,0.389377,256.0,12.0,54.0,35.0,9.781930e+06,0.152955,5.985915,0.872620,2,NaN
3,addressbook,crawl-addressbook-60min,index,state8,0.240000,0.159563,254.0,0.0,5.0,0.0,3.160270e+05,0.005957,30.851064,0.963849,2,NaN
4,addressbook,crawl-addressbook-60min,index,state10,0.534722,0.275247,301.0,0.0,11.0,4.0,4.475724e+04,0.032304,21.088435,0.931409,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8510,addressbook,crawl-addressbook-60min,state718,state724,0.019112,0.028763,2.0,0.0,38.0,17.0,2.370219e+03,0.045766,7.084020,0.721924,1,additionalRows
8511,addressbook,crawl-addressbook-60min,state718,state727,0.028397,0.042417,1.0,0.0,36.0,15.0,3.134268e+03,0.062806,6.729289,0.721729,1,additionalRows
8512,addressbook,crawl-addressbook-60min,state721,state724,0.009466,0.014403,2.0,0.0,39.0,13.0,1.981234e+03,0.026118,6.872205,0.721425,1,additionalRows
8513,addressbook,crawl-addressbook-60min,state721,state727,0.018754,0.028292,1.0,0.0,37.0,15.0,2.357432e+03,0.043811,7.145057,0.720557,1,additionalRows


In [ ]:
images_folder: str = "labeled-subject-set/GroundTruthModels-SS/addressbook/screenshots"
save_folder: str = "labeled-subject-set/GroundTruthModels-SS/addressbook/screenshots/concat"

for index, row in tqdm(addressbook_df.iterrows()):

  state1_thumb = f"{images_folder}/{row['state1']}_thumbnail.png"
  state2_thumb = f"{images_folder}/{row['state2']}_thumbnail.png"

  output_path = f"{save_folder}/concat_{row['state1']}_{row['state2']}.png"

  concat_images_with_line(state1_thumb, state2_thumb, output_path)


8515it [06:14, 22.74it/s]


# LLM

In [1]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch

from PIL import Image
import requests
import base64

import pandas as pd

from tqdm import tqdm

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
model = LlavaNextForConditionalGeneration.from_pretrained(
    "llava-hf/llava-v1.6-mistral-7b-hf",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)
model.to(device)

preprocessor_config.json:   0%|          | 0.00/754 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.85k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

LlavaNextForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): Q

In [8]:
# Paths to resources
concat_images_folder = "labeled-subject-set/GroundTruthModels-SS/addressbook/screenshots/concat"
html_content_folder = "labeled-subject-set/GroundTruthModels-SS/addressbook"

# Check and initialize the 'generated_text' column safely
if 'generated_text' not in addressbook_df.columns:
    addressbook_df['generated_text'] = None  # Initialize column with None

# Iterate over the DataFrame rows
for index, row in tqdm(addressbook_df.iterrows()):
    if pd.isna(row['generated_text']) and row['appname'] == 'addressbook':
        # Construct paths to the concatenated image and the HTML files
        image_path = f"{concat_images_folder}/concat_{row['state1']}_{row['state2']}.png"
        html_path1 = f"{html_content_folder}/{row['state1']}.html.content_tags"
        html_path2 = f"{html_content_folder}/{row['state2']}.html.content_tags"

        # Load image and HTML content
        image = Image.open(image_path)
        with open(html_path1, 'r') as file1, open(html_path2, 'r') as file2:
            html_content1 = file1.read()
            html_content2 = file2.read()

        # Create a prompt for the image and HTML content
        prompt = (f"[INST] <image>\nYou are tasked with evaluating two webpages displayed side-by-side in an image. "
                  f"Consider their visual appearance and the following HTML structures:\n"
                  f"HTML 1: {html_content1}\nHTML 2: {html_content2}\n"
                  "Your goal is to determine if these pages are simply different states of the same webpage or if they are nearly identical, "
                  "which impacts the generation of test cases for end-to-end testing. "
                  "Respond only with one of the following terms based on your assessment: distinct, near-duplicate, or clone. [/INST]")

        # Process the image and prompt
        inputs = processor(prompt, image, return_tensors="pt").to(device)
        output = model.generate(**inputs, max_new_tokens=10)
        decoded_output = processor.decode(output[0], skip_special_tokens=True)

        # Filter out the instruction text
        instruction_start = decoded_output.find("[INST]")
        instruction_end = decoded_output.find("[/INST]") + len("[/INST]")

        filtered_output = decoded_output[instruction_end:] if instruction_start != -1 and instruction_end != -1 else decoded_output

        # Append the filtered output to the DataFrame
        addressbook_df.at[index, 'generated_text'] = filtered_output
    if index == 100:
      break

0it [00:00, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
1it [00:03,  3.01s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
2it [00:04,  2.28s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
3it [00:06,  2.15s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
4it [00:08,  1.97s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
5it [00:10,  1.93s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
6it [00:12,  2.09s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
7it [00:14,  2.05s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
8it [00:16,  1.98s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
9it [00:18,  1.93s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
10it [00:20,  1.85s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
11it [00:21, 

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.57 GiB. GPU 0 has a total capacity of 22.17 GiB of which 1.41 GiB is free. Process 9568 has 20.75 GiB memory in use. Of the allocated memory 19.51 GiB is allocated by PyTorch, and 1.00 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [9]:
addressbook_df

,appname,crawl,state1,state2,DOM_RTED,DOM_Levenshtein,DOM_contentHash,DOM_SIMHASH,VISUAL_BlockHash,VISUAL_PHash,VISUAL_Hyst,VISUAL_PDiff,VISUAL_SIFT,VISUAL_SSIM,HUMAN_CLASSIFICATION,TAGS,generated_text
0,addressbook,crawl-addressbook-60min,index,state3,0.214286,0.148426,250.0,0.0,5.0,0.0,3.327536e+05,0.005017,54.166667,0.973036,2,NaN,distinct
1,addressbook,crawl-addressbook-60min,index,state5,0.622449,0.294496,329.0,12.0,44.0,34.0,1.690174e+07,0.149702,6.788512,0.877429,2,NaN,distinct
2,addressbook,crawl-addressbook-60min,index,state6,0.618182,0.389377,256.0,12.0,54.0,35.0,9.781930e+06,0.152955,5.985915,0.872620,2,NaN,distinct
3,addressbook,crawl-addressbook-60min,index,state8,0.240000,0.159563,254.0,0.0,5.0,0.0,3.160270e+05,0.005957,30.851064,0.963849,2,NaN,distinct
4,addressbook,crawl-addressbook-60min,index,state10,0.534722,0.275247,301.0,0.0,11.0,4.0,4.475724e+04,0.032304,21.088435,0.931409,2,NaN,distinct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8510,addressbook,crawl-addressbook-60min,state718,state724,0.019112,0.028763,2.0,0.0,38.0,17.0,2.370219e+03,0.045766,7.084020,0.721924,1,additionalRows,None
8511,addressbook,crawl-addressbook-60min,state718,state727,0.028397,0.042417,1.0,0.0,36.0,15.0,3.134268e+03,0.062806,6.729289,0.721729,1,additionalRows,None
8512,addressbook,crawl-addressbook-60min,state721,state724,0.009466,0.014403,2.0,0.0,39.0,13.0,1.981234e+03,0.026118,6.872205,0.721425,1,additionalRows,None
8513,addressbook,crawl-addressbook-60min,state721,state727,0.018754,0.028292,1.0,0.0,37.0,15.0,2.357432e+03,0.043811,7.145057,0.720557,1,additionalRows,None


In [10]:
addressbook_df.to_csv("addressbook3.csv")